In [1]:
import pandas as pd

In [2]:
products_df = pd.read_csv('../data/products.csv')
products_df.drop(['created_at', 'updated_at', 'deleted_at'], axis=1, inplace=True)
products_df.head()

,id,name,description,category,price,stock
0,1,T-shirt,A simple and comfortable T-shirt,1,20,100
1,2,Jeans,Classic blue jeans,1,50,50
2,3,Sneakers,Sporty and stylish sneakers,1,80,30
3,4,Hoodie,A warm and cozy hoodie,1,40,80
4,5,Running Shoes,Lightweight and comfortable shoes for running,1,70,20


In [3]:
orders_df = pd.read_csv('../data/orders.csv')
orders_df.drop(['created_at', 'updated_at', 'deleted_at'], axis=1, inplace=True)
orders_df.head()

,id,user,total,status,address
0,1,20,28000,2,1
1,2,19,40000,2,3
2,3,2,40000,3,3
3,4,3,200000,3,4
4,5,5,30000,3,5


In [5]:
order_items_df = pd.read_csv('../data/order_items.csv')
order_items_df.head()

,order,product,quantity,price
0,1,1,2,2000
1,1,2,1,5000
2,1,3,1,8000
3,1,4,1,4000
4,1,5,1,7000
